In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import onnx
import keras2onnx
import random
from es import SimpleGA, OpenES

In [ ]:
input_shape = (15,15,2)
input_len = input_shape[0] * input_shape[1] * input_shape[2]
model_name_onnx = "model4light.onnx"

In [ ]:
"""
## Build the model
"""
print("input shape=" + str(input_shape))
"""model = keras.Sequential(
    [
        keras.Input(shape=input_len, name="input"),
        layers.Reshape(input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        #layers.Dropout(0.5),
        layers.Dense(10, activation="relu"),
        layers.Dense(1, activation="sigmoid", name="output")
    ]
)"""
"""
model = keras.Sequential()
model.add(layers.Dense(20, activation='sigmoid', input_shape = input_shape))
model.add(layers.Dense(20, activation='sigmoid'))
model.add(layers.Dense(num_classes, activation="softmax"))
"""

def create_model():
    return keras.Sequential(
        [
            keras.Input(shape=input_len, name="input"),
            layers.Reshape(input_shape),
            layers.Conv2D(16, kernel_size=(5, 5), activation="relu"),
            layers.Conv2D(8, kernel_size=(3, 3), activation="relu"),
            layers.Flatten(),
            #layers.Dropout(0.5),
            layers.Dense(2, activation="relu"),
            layers.Dense(1, activation="sigmoid", name="output")
        ]
    )
model = create_model()

model.summary()

In [ ]:
model.compile()

In [ ]:
onnx_model = keras2onnx.convert_keras(model, model.name)

onnx.save_model(onnx_model, model_name_onnx)

In [ ]:
random.seed(1)
field_count = 25
random_fields = np.zeros((field_count,input_shape[0],input_shape[1],input_shape[2]), dtype="float32")
print(random_fields.shape)
for field in range(field_count):
    for row in range(input_shape[0]):
        for col in range(input_shape[1]):
            r = random.randint(0,3);
            if(r == 0 or r == 1):
                random_fields[field][row][col][r] = 1

random_fields_flat = random_fields.reshape((field_count,input_len))
#print(random_fields_flat.shape)
#for field in random_fields_flat:
#    print(field)

print(model.predict(random_fields_flat,verbose=1))

In [ ]:
model_weights = model.get_weights()
### unwrap layers weights
layer_count = len(model_weights)
print("layer count: " + str(layer_count))
layer_shapes = [None] * layer_count
flat_layers = [None] * layer_count
layer_n = 0;
for layer in model_weights:
    np_layer = np.array(layer)
    layer_shapes[layer_n] = np_layer.shape
    flat_layers[layer_n] = np_layer.reshape(-1)
    layer_n += 1

    
print("layer shapes: " + str(layer_shapes))
print("layer sizes: ");
for layer in model_weights:
    print(layer.size)
    
flat_weights = np.concatenate(flat_layers)
print("weight count: " + str(len(flat_weights)))
print("src weights:")
for i in range(20):
    print(flat_weights[i])

In [ ]:
##### manipulate layer weights

for i in range(1, len(flat_weights), 1):
    flat_weights[i] = 0
    
print("manipulated weights:")
for i in range(20):
    print(flat_weights[i])
    

In [ ]:
#### wrap layer weights back
split_indexes = [len(l) for l in flat_layers]
print(split_indexes)
for i in range(1, len(split_indexes)):
    split_indexes[i] += split_indexes[i-1]
print(split_indexes)
flat_layers = np.split(flat_weights, split_indexes)
flat_layers.pop()
layer_n = 0
for layer in flat_layers:
    model_weights[layer_n] = layer.reshape(layer_shapes[layer_n])
    layer_n += 1

model.set_weights(model_weights)

In [ ]:
NPARAMS = sum([np.array(layer).size for layer in model.get_weights()]) # make this a n-dimensinal problem.
print("NPARAMS: " + str(NPARAMS))
NPOPULATION = 16 # population size .
MAX_ITERATION = 40 # run each solver for n generations.

In [ ]:
import gomokutrainer
import importlib
importlib.reload(gomokutrainer)
evaluator = gomokutrainer.Evaluator(create_model, "../EstimatorExe/OnnxEstimator.exe")
oes = OpenES(NPARAMS,                  # number of model parameters
            sigma_init=0.5,            # initial standard deviation
            sigma_decay=0.999,         # don't anneal standard deviation
            learning_rate=0.1,         # learning rate for standard deviation
            learning_rate_decay = 1.0, # annealing the learning rate
            popsize=NPOPULATION,       # population size
            antithetic=False,          # whether to use antithetic sampling
            weight_decay=0.00,         # weight decay coefficient
            rank_fitness=False,        # use rank rather than fitness numbers
            forget_best=False)
list_of_flat_weights = oes.ask()
print("list of flat weights len: " + str(len(list_of_flat_weights)))
eval_result = evaluator.evaluate(list_of_flat_weights, "C:\Projects\Testing")
print("evaluation result: " + str(eval_result))



In [ ]:
def test_solver(solver):
    history = []
    for j in range(MAX_ITERATION):
        list_of_flat_weights = solver.ask()
        eval_result = evaluator.evaluate(list_of_flat_weights, "C:\Projects\Testing2")
        solver.tell(eval_result)
        result = solver.result() # first element is the best solution, second element is the best fitness
        history.append(result[1])
    return history

In [ ]:
oes = OpenES(NPARAMS,                  # number of model parameters
            sigma_init=0.5,            # initial standard deviation
            sigma_decay=0.999,         # don't anneal standard deviation
            learning_rate=0.1,         # learning rate for standard deviation
            learning_rate_decay = 1.0, # annealing the learning rate
            popsize=NPOPULATION,       # population size
            antithetic=False,          # whether to use antithetic sampling
            weight_decay=0.00,         # weight decay coefficient
            rank_fitness=True,        # use rank rather than fitness numbers
            forget_best=True)

In [ ]:
oes_history = test_solver(oes)

In [ ]:
ga = SimpleGA(NPARAMS,                # number of model parameters
               sigma_init=0.5,        # initial standard deviation
               popsize=NPOPULATION,   # population size
               elite_ratio=0.1,       # percentage of the elites
               forget_best=False,     # forget the historical best elites
               weight_decay=0.00,     # weight decay coefficient
              )

In [ ]:
ga_history = test_solver(ga)